In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import random as rn
from datetime import datetime as dt
from datetime import timedelta
import bnlearn as bn


%run Functions.ipynb


In [ ]:
df = pd.read_csv("ToyDataset.csv").drop(['Unnamed: 0'],axis=1)
df["EntryDate"] = pd.to_datetime(df['EntryDate'])
df["ExitDate"] = pd.to_datetime(df['ExitDate'])
df = df.sort_values(["EntryDate",'ExitDate']).reset_index(drop=True)

data = df[['ClientID','EntryDate','ExitDate','Destination','ProjectType','ProjectID']]



In [ ]:
accuracy_df = pd.DataFrame()
precision_recall_df = pd.DataFrame()
class_PR = pd.DataFrame()
pred = pd.DataFrame()
K=3
bnlearn_data = data_prep(data,K)
n = 1
test_length = int(len(data)*0.2)
test = bnlearn_data.iloc[0:test_length]
train = bnlearn_data.drop(test.index)  



# Model
print('Model')
DAG = parameter_learning(train,K)
prediction = predict(test,DAG,K)
prediction_labelled = prediction.copy()
for a in np.sort(prediction['Predicted pK'].unique()): 
    prediction_labelled['Predicted pK'].replace(a,np.sort(train['PT'+str(K+1)].unique())[int(a)],inplace=True )       
accuracy_df = accuracy_df.append({"Model":"PREVISE","K":K,"CV":n,"Accuracy for exit":accuracy(prediction_labelled)[0],"Accuracy for pK":accuracy(prediction_labelled)[1]},ignore_index=True)
PR_pK = recall_precision_pK(prediction_labelled,'PREVISE')
PR_exit = recall_precision_exit(prediction_labelled,'PREVISE')
PR_pK['CV'] = [n for a in range(len(PR_pK))]
PR_exit['CV'] = [n for a in range(len(PR_exit))]
PR_pK['Class type'] = ['pK' for a in range(len(PR_pK))]
PR_exit['Class type'] = ['Exit' for a in range(len(PR_exit))]
class_PR = class_PR.append(PR_pK)
class_PR = class_PR.append(PR_exit)
recall_pK = PR_pK.groupby(['K','Model']).mean().drop('Class',axis=1)['Recall'].values[0]
recall_exit = PR_exit.groupby(['K','Model']).mean().drop('Class',axis=1)['Recall'].values[0]
precision_pK = PR_pK.groupby(['K','Model']).mean().drop('Class',axis=1)['Precision'].values[0]
precision_exit = PR_exit.groupby(['K','Model']).mean().drop('Class',axis=1)['Precision'].values[0]
precision_recall_df = precision_recall_df.append({"Model":"PREVISE","K":K,"CV":n,"Recall for exit":recall_exit,"Recall for pK":recall_pK,"Precision for exit":precision_exit,"Precison for pK":precision_pK},ignore_index=True)


